**CSC-555**

**Restaurant-Recommendation**

In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1731966853693_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("numpy")
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1731966853693_0001/container_1731966853693_0001_01_000001/tmp/spark-b51ea24e-9a68-4fdb-864a-d6005d1e992e
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.15.30 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [3]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Similar Restaurants") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Loading the final file after aggregation through athena
final_restaurants_path = "s3://bucket-akshat-test/restaurants_final/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load the final aggregated dataset
restaurants_final_df = spark.read.parquet(final_restaurants_path)
print("Final Restaurants DataFrame Schema:")
restaurants_final_df.printSchema()
restaurants_final_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final Restaurants DataFrame Schema:
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- avg_menu_price: double (nullable = true)
 |-- unique_menu_categories: long (nullable = true)

+-----+--------------------+----------+-----------+------------------+----------------------+
|   id|                name|       lat|        lng|    avg_menu_price|unique_menu_categories|
+-----+--------------------+----------+-----------+------------------+----------------------+
|40891|Denny's (1601 I35 N)|30.2767055|-97.7296076| 7.333944970314656|                    16|
|    5|Nelson Brothers C...|  33.51473|   -86.8117| 4.464925374557723|                     9|
|20418|     Taco City DC NW|  38.92385|  -77.05205| 6.944186046373012|                     5|
|20426| Mai Thai (Old Town)|38.8040633|-77.0399785|14.356250002980232|                    10|
|45048|Luby's (2695 Inte...|30.0948838|-94.1293961| 8.3099999

In [7]:
# Filter out rows with null values
r_data = restaurants_final_df.filter(
    col("lat").isNotNull() & 
    col("lng").isNotNull() & 
    col("avg_menu_price").isNotNull()
)
features = r_data.select("id", "name", "lat", "lng", "avg_menu_price")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Combine features into a single vector
vector_assembler = VectorAssembler(inputCols=["lat", "lng", "avg_menu_price"], outputCol="features")
vector_data = vector_assembler.transform(features)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Normalize the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(vector_data)
normalized_data = scaler_model.transform(vector_data)

# Show normalized data
normalized_data.select("scaled_features").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------+
|scaled_features                                                |
+---------------------------------------------------------------+
|[-0.9785442703954408,-0.06976821530038146,-0.29439372489156124]|
|[-0.47701727684485073,0.6864722700679267,-0.7272216973740957]  |
|[0.36078357714941867,1.3624848105133889,-0.35319379898520936]  |
|[0.34223020810655136,1.363320955803638,0.7650100010145496]     |
|[-1.0067060371032763,0.17960426447665628,-0.14714345699944062] |
+---------------------------------------------------------------+
only showing top 5 rows

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
from pyspark.sql import Window
from pyspark.sql.functions import row_number, collect_list, struct
import random
from pyspark.sql.functions import rand

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 1. Define a UDF to calculate cosine similarity between two vectors

In [ ]:
def cosine_similarity_udf(vector1, vector2):
    dot_product = float(vector1.dot(vector2))  # Using Spark's MLlib dot product method
    magnitude1 = float(vector1.norm(2))  # L2 norm (Euclidean norm)
    magnitude2 = float(vector2.norm(2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    return dot_product / (magnitude1 * magnitude2)

# Register the UDF
cosine_similarity = udf(cosine_similarity_udf, DoubleType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2. Compare each restaurant with every other restaurant (using cross join)

In [ ]:
similarities = normalized_data.alias("a").crossJoin(normalized_data.alias("b")) \
    .withColumn("similarity", cosine_similarity(col("a.scaled_features"), col("b.scaled_features"))) \
    .filter(col("a.id") != col("b.id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Sample random restaurants from the dataset
sampled_data = normalized_data.orderBy(rand()).limit(5)

# Get the list of IDs and names for the sampled restaurants
sampled_data_with_names = sampled_data.select("id", "name")

# Show the sampled restaurants with ID and name
sampled_data_with_names.show(truncate=False)
sample_ids = sampled_data.select("id").rdd.flatMap(lambda x: x).collect()

# Filter the similarities DataFrame to only include those for the sampled restaurants
sample_similarities = similarities.filter(col("a.id").isin(sample_ids))

# Rank the similarities within each restaurant by similarity score 
window_spec = Window.partitionBy("a.id").orderBy(col("similarity").desc())
ranked_sample_similarities = sample_similarities.withColumn("rank", row_number().over(window_spec))

# Filter to top 5 similar restaurants for each sampled restaurant
top_5_sample_similarities = ranked_sample_similarities.filter(col("rank") <= 5)

# Format the output to show similar restaurants for each sampled restaurant
formatted_sample_output = top_5_sample_similarities.select(
    col("a.id").alias("Restaurant ID"),
    col("b.name").alias("Similar Restaurant"),
    col("similarity").alias("Score")
)

# Show the results for the sampled restaurants
formatted_sample_output.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------------------------+
|id   |name                                  |
+-----+--------------------------------------+
|23922|Pasta Americana (7425 Bell Creek Road)|
|61332|El Pollo Loco (601 Nolana Loop,3463)  |
|62587|Las Palapas (1675 W State)            |
|33457|Rocco's Italian Cafe                  |
|42309|Ace Liquor                            |
+-----+--------------------------------------+

+-------------+----------------------------------------------------+------------------+
|Restaurant ID|Similar Restaurant                                  |Score             |
+-------------+----------------------------------------------------+------------------+
|23922        |Smokey Bones (Roanoke)                              |0.9998595766318839|
|23922        |Caribbean Wraps Spot (104 Whitehall Rd Unit #1B)    |0.9998402671277096|
|23922        |sweetFrog (10040 Sliding Hill Rd)                   |0.9998204165040929|
|23922        |Hunt Brothers Pizza              


- RECOMMENDATIONS FOR ENTIRE DATASET CANNOT BE PERFORMED AS THE DATASET IS TOO BIG
CONSIDERED SAMPLE OF 500 instead


In [ ]:
# Sample 500 random restaurants from the dataset
sampled_data = normalized_data.sample(False, 500 / normalized_data.count())  # Adjust sample size as needed
sample_ids = sampled_data.select("id").rdd.flatMap(lambda x: x).collect()

# Filter the similarities for only the sampled restaurants
sample_similarities = similarities.filter(col("a.id").isin(sample_ids))

# Rank by similarity within each restaurant
window_spec = Window.partitionBy("a.id").orderBy(col("similarity").desc())
ranked_sample_similarities = sample_similarities.withColumn("rank", row_number().over(window_spec))

# Filter to top 5 for each sample restaurant
top_5_sample_similarities = ranked_sample_similarities.filter(col("rank") <= 5)

formatted_sample_output = top_5_sample_similarities.select(
    col("a.id").alias("Restaurant ID"),
    col("a.name").alias("Restaurant Name"),
    col("b.name").alias("Similar Restaurant"),
    col("similarity").alias("Score")
)

formatted_sample_output.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------------------------------------+----------------------------------------------------+------------------+
|Restaurant ID|Restaurant Name                              |Similar Restaurant                                  |Score             |
+-------------+---------------------------------------------+----------------------------------------------------+------------------+
|86           |SOCU Southern Kitchen + Oyster Bar Birmingham|Michael's Steaks &amp; Seafood                      |0.999851554633651 |
|86           |SOCU Southern Kitchen + Oyster Bar Birmingham|Taziki's Mediterranean Café (2737 Highway 280 South)|0.9996712006692721|
|86           |SOCU Southern Kitchen + Oyster Bar Birmingham|Ocean Restaurant                                    |0.9995828507300235|
|86           |SOCU Southern Kitchen + Oyster Bar Birmingham|Panda Express (3780 Lorna Rd)                       |0.9994996516877626|
|86           |SOCU Southern Kitchen + Oyster Bar Birmingham|P